In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import main_constants as ct
import json
from services.index import Index
import unidecode
from tqdm import tqdm
from retrieval.feature_extractors.EntityExtractor import EntityExtractor
import torch
from services import sql
import sqlite3
import collections as cl
import json
import pickle
import pandas as pd
import spacy
import numpy as np

In [ ]:
# check if embeddings are right.
from retrieval.neural.modules.encoders import MaxPoolEncoder, MeanPoolEncoder
from retrieval.neural.modules.scorers import LinearLogisticRegression
encoder = MaxPoolEncoder(50)
encoder.embed(torch.tensor(1).view(1,1))

In [ ]:
# print term frquencies to see if they are ordered.
with open('./data/index/token2id.tar', 'rb') as file:
    t2i = pickle.load(file)
with open('./data/index/id2tf.tar', 'rb') as file:
    i2tf = pickle.load(file)
for t in list(t2i.keys())[0:100]:
    print(f'{t}\t\t{t2i[t]}\t{i2tf[t2i[t]]}')

In [ ]:
# time how fast pyndri can query: very slow...
index = Index('tfidf')
# as opposed to db lookup.
def fn():
    with sqlite3.connect('./data/term_retrievals/uni_tfidf.dummy/retrievals.sqlite') as db:
        (q_id, target_titles, doc_int_ids) = db.cursor() \
            .execute(sql.get_question_by_qid(), ('5a7a52745542996c55b2dd4f',)) \
            .fetchone()
        return pickle.loads(doc_int_ids)[:1000]
%timeit index.unigram_query('What is anarchism?', 1000)
%timeit fn()

In [ ]:
# check run looks good.
run = Run()
run.add_ranking("a", {"a1": 1})
run.add_ranking("b", {"b1": 1, "b2": 2})
run.update_ranking("d", "d1", 3)
run.update_rankings({"c": {"c1": 3}, "b": {"b3": 3}})
run

In [24]:
df = pd.read_pickle('./data/candidates/tfidf.train.dummy.gzip', compression='gzip')
len(df)

4

In [4]:
ent_model = spacy.load('xx_ent_wiki_sm')
index = Index()
db = sqlite3.connect(ct.DOCUMENT_DB)
cursor = db.cursor()
(doc_1,) = cursor.execute("SELECT text FROM documents WHERE id = ?", (index.title2wid["Scott Derrickson"],)).fetchone()
(doc_2,) = cursor.execute("SELECT text FROM documents WHERE id = ?", (index.title2wid["Ed Wood"],)).fetchone()
question = "Were Scott Derrickson and Ed Wood of the same nationality?"
cursor.close()
db.close()
question_parse = ent_model(question)
doc_1_parse = ent_model(doc_1)
doc_2_parse = ent_model(doc_2)

INFO:root:[2018-12-10 22:54:14.448095]	[5189]	Loading index ./data/index/indri with default query environment.
INFO:root:[2018-12-10 22:54:18.897701]	[5189]	Not loading stopwords.
INFO:root:[2018-12-10 22:54:26.300100]	[5189]	Loaded index in 0:00:11.851131.


In [5]:
spacy.displacy.render([question_parse, doc_1_parse, doc_2_parse], style='ent', jupyter=True)

In [6]:
ents_question = [(e.text, e.start_char, e.end_char, e.label_) for e in question_parse.ents]
ents_question

[('Scott Derrickson', 5, 21, 'PER'), ('Ed Wood', 26, 33, 'PER')]

In [7]:
ents_doc_1 = [(e.text, e.start_char, e.end_char, e.label_) for e in doc_1_parse.ents]
ents_doc_1

[('Scott Derrickson', 0, 16, 'PER'),
 ('American', 44, 52, 'MISC'),
 ('He lives', 97, 105, 'MISC'),
 ('Los Angeles', 109, 120, 'LOC'),
 ('California', 123, 133, 'LOC'),
 ('Sinister', 197, 205, 'MISC'),
 ('The Exorcism of Emily Rose', 210, 236, 'MISC'),
 ('Deliver Us From Evil', 245, 265, 'MISC'),
 ('Marvel Cinematic Universe', 289, 314, 'ORG'),
 ('Doctor Strange', 330, 344, 'MISC'),
 ('Derrickson', 354, 364, 'PER'),
 ('Denver', 376, 382, 'LOC'),
 ('Colorado', 384, 392, 'LOC'),
 ('Biola University', 420, 436, 'ORG'),
 ('Humanities', 454, 464, 'LOC'),
 (' He', 614, 617, 'PER'),
 ('M.A.  ', 629, 635, 'PER'),
 ('USC School of Cinematic Arts', 659, 687, 'ORG')]

In [40]:
ents_doc_2 = [(e.text, e.start_char, e.end_char, e.label_) for e in doc_2_parse.ents]
ents_doc_2

[('Edward Davis Wood Jr.', 0, 21, 'PER'),
 ('American', 68, 76, 'MISC'),
 ('In the 1950s', 133, 145, 'MISC'),
 ('Wood', 147, 151, 'PER'),
 ('Golden Turkey Award', 416, 435, 'MISC'),
 ('Worst Director of All Time', 440, 466, 'MISC'),
 ('Wood', 513, 517, 'PER')]

In [8]:
for token in doc_1_parse:
    if token.ent_type_:
        print([token.i, token.text, token.ent_iob_, token.ent_type_])

[0, 'Scott', 'B', 'PER']
[1, 'Derrickson', 'I', 'PER']
[11, 'American', 'B', 'MISC']
[20, 'He', 'B', 'MISC']
[21, 'lives', 'I', 'MISC']
[23, 'Los', 'B', 'LOC']
[24, 'Angeles', 'I', 'LOC']
[26, 'California', 'B', 'LOC']
[42, 'Sinister', 'B', 'MISC']
[46, 'The', 'B', 'MISC']
[47, 'Exorcism', 'I', 'MISC']
[48, 'of', 'I', 'MISC']
[49, 'Emily', 'I', 'MISC']
[50, 'Rose', 'I', 'MISC']
[55, 'Deliver', 'B', 'MISC']
[56, 'Us', 'I', 'MISC']
[57, 'From', 'I', 'MISC']
[58, 'Evil', 'I', 'MISC']
[66, 'Marvel', 'B', 'ORG']
[67, 'Cinematic', 'I', 'ORG']
[68, 'Universe', 'I', 'ORG']
[73, 'Doctor', 'B', 'MISC']
[74, 'Strange', 'I', 'MISC']
[78, 'Derrickson', 'B', 'PER']
[82, 'Denver', 'B', 'LOC']
[84, 'Colorado', 'B', 'LOC']
[91, 'Biola', 'B', 'ORG']
[92, 'University', 'I', 'ORG']
[99, 'Humanities', 'B', 'LOC']
[129, ' ', 'B', 'PER']
[130, 'He', 'I', 'PER']
[133, 'M.A.', 'B', 'PER']
[134, ' ', 'I', 'PER']
[139, 'USC', 'B', 'ORG']
[140, 'School', 'I', 'ORG']
[141, 'of', 'I', 'ORG']
[142, 'Cinematic', 'I',

In [9]:
question_ents_dict = cl.defaultdict(list)
for (entity, _, _, _type) in ents_question:
    question_ents_dict[_type].append(entity)
doc_ents_dict = cl.defaultdict(list)
for (entity, _, _, _type) in ents_doc_1:
    doc_ents_dict[_type].append(entity)
question_ents_dict, doc_ents_dict

(defaultdict(list, {'PER': ['Scott Derrickson', 'Ed Wood']}),
 defaultdict(list,
             {'PER': ['Scott Derrickson', 'Derrickson', ' He', 'M.A.  '],
              'MISC': ['American',
               'He lives',
               'Sinister',
               'The Exorcism of Emily Rose',
               'Deliver Us From Evil',
               'Doctor Strange'],
              'LOC': ['Los Angeles',
               'California',
               'Denver',
               'Colorado',
               'Humanities'],
              'ORG': ['Marvel Cinematic Universe',
               'Biola University',
               'USC School of Cinematic Arts']}))

In [38]:
entity_type_matching_word_count = cl.defaultdict(int)
doc_entity_type_total_word_count = 0
for _type, question_entities in question_ents_dict.items():
    for doc_entity in doc_ents_dict[_type]:
        for doc_entity_word in doc_entity.strip().split():
            doc_entity_type_total_word_count += 1
            doc_entity_word = doc_entity_word.lower()
            for question_entity in question_entities:
                for question_entity_word in question_entity.strip().split():
                    question_entity_word = question_entity_word.lower()
                    entity_type_matching_word_count[_type] += int(question_entity_word == doc_entity_word)

    print(_type, entity_type_matching_word_count[_type], doc_entity_type_total_word_count)
    entity_type_matching_word_count[_type] = entity_type_matching_word_count[_type] / doc_entity_type_total_word_count
dict(entity_type_matching_word_count)

PER 3 5


{'PER': 0.6}

In [45]:
question_ents_dict = cl.defaultdict(list)
for (entity, _, _, _type) in ents_question:
    question_ents_dict[_type].append(entity)
doc_ents_dict = cl.defaultdict(list)
for (entity, _, _, _type) in ents_doc_2:
    doc_ents_dict[_type].append(entity)
question_ents_dict, doc_ents_dict

(defaultdict(list, {'PER': ['Scott Derrickson', 'Ed Wood']}),
 defaultdict(list,
             {'PER': ['Edward Davis Wood Jr.', 'Wood', 'Wood'],
              'MISC': ['American',
               'In the 1950s',
               'Golden Turkey Award',
               'Worst Director of All Time']}))

In [46]:
entity_type_matching_word_count = cl.defaultdict(int)
doc_entity_type_total_word_count = 0
for _type, question_entities in question_ents_dict.items():
    for doc_entity in doc_ents_dict[_type]:
        for doc_entity_word in doc_entity.strip().split():
            doc_entity_type_total_word_count += 1
            doc_entity_word = doc_entity_word.lower()
            for question_entity in question_entities:
                for question_entity_word in question_entity.strip().split():
                    question_entity_word = question_entity_word.lower()
                    entity_type_matching_word_count[_type] += int(question_entity_word == doc_entity_word)

    print(_type, entity_type_matching_word_count[_type], doc_entity_type_total_word_count)
    entity_type_matching_word_count[_type] = entity_type_matching_word_count[_type] / doc_entity_type_total_word_count
dict(entity_type_matching_word_count)

PER 3 6


{'PER': 0.5}

In [47]:
e2i = {
    "PER": 0,
    "LOC": 1,
    "ORG": 2,
    "MISC": 3
}
qu_ents = cl.defaultdict(list)
for (entity, _, _, _type) in ents_question:
    qu_ents[_type].append(entity)
doc_ents = cl.defaultdict(list)
for (entity, _, _, _type) in ents_doc_2:
    doc_ents[_type].append(entity)
question_ents_dict, doc_ents_dict

(defaultdict(list, {'PER': ['Scott Derrickson', 'Ed Wood']}),
 defaultdict(list,
             {'PER': ['Edward Davis Wood Jr.', 'Wood', 'Wood'],
              'MISC': ['American',
               'In the 1950s',
               'Golden Turkey Award',
               'Worst Director of All Time']}))

[3. 0. 0. 0.] [6. 0. 0. 0.]


array([0.5, 0. , 0. , 0. ])

In [3]:
df = pd.read_pickle(ct.DOCUMENT_DB, compression='gzip')

In [6]:
len(df)

5486212

In [3]:
index = Index('tfidf')

In [4]:
%timeit index.unigram_query('What is Anarchism?', request=1000)

1.07 s ± 38.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
a = [1,2,3]

In [8]:
a[0:2]

[1, 2]

In [9]:
import collections as cl

In [11]:
a = cl.Counter((1,1,1,1,2,2,3,5,6,8))

In [13]:
1 in a

True

In [14]:
9 in a

False

In [15]:
for token_counts in a:
    print(token_counts)

1
2
3
5
6
8


In [4]:
index = Index('tfidf')
ext = EntityExtractor(index)

In [5]:
db = sqlite3.connect('./data/features/train.dummy.feature.db')
cursor= db.cursor()

result = cursor.execute("""SELECT * FROM features WHERE query_id ='"5adf732a5542993a75d264e9"'""").fetchall()

In [6]:
result[0]

('"5adf732a5542993a75d264e9"',
 '"bridge"',
 '"easy"',
 '939821',
 '41539372',
 '"Kelli Ward"',
 '"Which  American politician did Donahue replaced "',
 '"Kelli Ward (\\"nee\\" Kaznoski; born January 25, 1969) is an American politician  and osteopathic physician . 0eos0  A member of the Republican Party , Ward was elected in 2012 to serve in the Arizona State Senate  representing the fifth legislative district. 0eos0  She was unopposed for election in 2014. 0eos0  While serving in office, she continued to practice medicine in the emergency departments in Lake Havasu City  and Kingman , Arizona. 0eop0 In July 2015, Ward announced her candidacy for the United States Senate , challenging incumbent U.S. Senator John McCain  in the Republican primary  in the 2016 election . 0eos0  That December, she resigned from the State Senate in order to focus her efforts on the U.S. Senate race. 0eos0  In August 2016 she was defeated by McCain in the Republican primary, 51 percent to 39 percent."',
 '[2

In [11]:
query_text = json.loads(result[0][6])
doc_text = json.loads(result[0][7])

In [13]:
query_text

'Which  American politician did Donahue replaced '

In [14]:
doc_text

'Kelli Ward ("nee" Kaznoski; born January 25, 1969) is an American politician  and osteopathic physician . 0eos0  A member of the Republican Party , Ward was elected in 2012 to serve in the Arizona State Senate  representing the fifth legislative district. 0eos0  She was unopposed for election in 2014. 0eos0  While serving in office, she continued to practice medicine in the emergency departments in Lake Havasu City  and Kingman , Arizona. 0eop0 In July 2015, Ward announced her candidacy for the United States Senate , challenging incumbent U.S. Senator John McCain  in the Republican primary  in the 2016 election . 0eos0  That December, she resigned from the State Senate in order to focus her efforts on the U.S. Senate race. 0eos0  In August 2016 she was defeated by McCain in the Republican primary, 51 percent to 39 percent.'

In [17]:
ent_model = spacy.load('xx_ent_wiki_sm')

In [52]:
e2i = {
    "PER": 0,
    "LOC": 1,
    "ORG": 2,
    "MISC": 3
}
i2e = {
    0: "PER",
    1: "LOC",
    2: "ORG",
    3: "MISC"
}
import re

def extract(question: str, doc: str):
    qu_ents = _get_entities(question)
    doc_ents = _get_entities(doc)

    entity_scores = _score_entities(qu_ents, doc_ents)

    return entity_scores.tolist()

def _get_entities(text: str):
    parse = ent_model(text)

    ents = cl.defaultdict(list)
    for ent in parse.ents:
        ents[ent.label_].append(ent.text)

    return dict(ents)

def _score_entities(qu_ents, doc_ents):
    regex = re.compile("\ +")
    ent_type_word_matches = np.zeros(len(e2i), dtype=np.float)
    doc_ent_type_word_count = np.zeros(len(e2i), dtype=np.float)
    for _type, _qu_ents in qu_ents.items():
        _id = e2i[_type]
        # a bit unnatural but loop first over the documents so I can get the count of words without double-counting.
        for _doc_ents in doc_ents.get(_type, []):
            for doc_ent_word in regex.split(_doc_ents.strip()):
                doc_ent_word = doc_ent_word.lower()
                # count the words in the document entities
                doc_ent_type_word_count[_id] += 1
                for _q_ent in _qu_ents:
                    for question_entity_word in regex.split(_q_ent.strip()):
                        question_entity_word = question_entity_word.lower()
                        print(doc_ent_word, question_entity_word)                        
                        ent_type_word_matches[_id] += int(question_entity_word == doc_ent_word)
                        doc_ent_word = doc_ent_word.lower()
    # do a safe division where 0s remain 0s.
    # Because of the way it is built, if the doc_count is 0 also the matches are also 0.
    return np.divide(ent_type_word_matches, doc_ent_type_word_count, where=doc_ent_type_word_count != 0)

In [53]:
extract(query_text, doc_text)

american american
republican american
republican american
lake donahue
havasu donahue
city donahue
and donahue
kingman donahue
arizona donahue
u.s. donahue
[0. 0. 0. 1.] [0. 7. 0. 3.]


[0.0, 0.0, 0.0, 0.3333333333333333]

In [20]:
question_parse = ent_model(query_text)
doc_parse = ent_model(doc_text)
spacy.displacy.render([question_parse, doc_parse], style='ent', jupyter=True)

Kelli Ward ("nee" Kaznoski; born January 25, 1969) is an American politician  and osteopathic physician . 0eos0  A member of the Republican Party , Ward was elected in 2012 to serve in the Arizona State Senate  representing the fifth legislative district. 0eos0  She was unopposed for election in 2014. 0eos0  While serving in office, she continued to practice medicine in the emergency departments in Lake Havasu City  and Kingman , Arizona. 0eop0 In July 2015, Ward announced her candidacy for the United States Senate , challenging incumbent U.S. Senator John McCain  in the Republican primary  in the 2016 election . 0eos0  That December, she resigned from the State Senate in order to focus her efforts on the U.S. Senate race. 0eos0  In August 2016 she was defeated by McCain in the Republican primary, 51 percent to 39 percent.

In [31]:
ext.extract(query_text, doc_text)

[5e-324, 0.0, 9.9e-322, 0.0]